In [1]:
import re
import os
import pandas as pd
from pandas_plink import read_plink1_bin

In [2]:
revision_folder = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/"

In [3]:
# load all results

In [4]:
# myfile0 = Marcs_dir+"results_mj/TestOut/qtl_results_all.txt.gz"
# df0 = pd.read_csv(myfile0, index_col=0, sep="\t", compression='gzip')

In [5]:
myfile0 = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/Marcs_results_all.csv"
df0 = pd.read_csv(myfile0)

In [6]:
df0.shape

(11644600, 22)

In [7]:
# df0.drop_duplicates()
# df0.shape

In [8]:
# compute z scores
df0['z'] = df0['beta'] / df0['beta_se']

In [9]:
df0.head()

,Unnamed: 0,feature_id,snp_id,p_value,beta,beta_se,empirical_feature_p_value,feature_chromosome,feature_start,feature_end,...,beta_param,rho,snp_chromosome,snp_position,assessed_allele,call_rate,maf,hwe_p,eqtl,z
0,1,ENSG00000221909,7_99141373_G_A,0.028874,0.055972,0.025613,0.463113,7,99143931,99156159,...,20.152657,0.8,7,99141373,A,1.0,0.075758,0.436055,ENSG00000221909-7_99141373_G_A,2.185301
1,2,ENSG00000221909,7_99024636_C_T,0.036647,0.051505,0.024647,0.545354,7,99143931,99156159,...,20.152657,0.8,7,99024636,T,1.0,0.106061,0.593926,ENSG00000221909-7_99024636_C_T,2.089756
2,3,ENSG00000221909,7_99030782_A_T,0.036647,0.051505,0.024647,0.545354,7,99143931,99156159,...,20.152657,0.8,7,99030782,T,1.0,0.106061,0.593926,ENSG00000221909-7_99030782_A_T,2.089756
3,4,ENSG00000221909,7_99032593_G_T,0.036647,0.051505,0.024647,0.545354,7,99143931,99156159,...,20.152657,0.8,7,99032593,T,1.0,0.106061,0.593926,ENSG00000221909-7_99032593_G_T,2.089756
4,5,ENSG00000221909,7_99034055_G_A,0.036647,0.051505,0.024647,0.545354,7,99143931,99156159,...,20.152657,0.8,7,99034055,A,1.0,0.106061,0.593926,ENSG00000221909-7_99034055_G_A,2.089756


In [10]:
## read in genotype file (plink format)
plink_folder = "/hps/nobackup/hipsci/scratch/genotypes/imputed/2017-03-27/Full_Filtered_SNPs_Plink/"
plink_file = plink_folder+"hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.bed"
G = read_plink1_bin(plink_file)

Mapping files: 100%|██████████| 3/3 [06:25<00:00, 128.62s/it]


In [11]:
input_files_dir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/new/input_files/"
sample_mapping_file = input_files_dir+"sample_mapping_file.csv"
sample_mapping = pd.read_csv(sample_mapping_file, dtype={"genotype_individual_id": str, "phenotype_sample_id": str})

In [12]:
samples = sample_mapping['genotype_individual_id'].unique()
len(samples)

126

In [13]:
G_sel = G[G['sample'].isin(samples),:]
G_sel.shape

/nfs/software/stegle/users/acuomo/conda-envs/struct-lmm2/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


(125, 10464962)

In [14]:
# consider a specific gene

In [15]:
# equivalent to grep
# import re
# re.search("ENSG00000211643",genes)

In [16]:
endo_eqtl_file = input_files_dir+"endodiff_eqtl_allconditions_FDR10pct.csv"
endo_eqtl = pd.read_csv(endo_eqtl_file, index_col = False)
genes = endo_eqtl['feature'].unique()

In [19]:
genes

array(['ENSG00000164587_RPS14', 'ENSG00000134809_TIMM10',
       'ENSG00000197728_RPS26', ..., 'ENSG00000132130_LHX1',
       'ENSG00000112559_MDFI', 'ENSG00000141012_GALNS'], dtype=object)

In [ ]:
for gene_name in genes:
    gene = re.sub("_.*","",gene_name)
    print(gene)
    if os.path.exists(revision_folder+"LD_matrices_for_susie/"+str(gene)+".csv"): continue
    df1 = df0[df0['feature_id']==gene]
    snps = df1['snp_id'].unique()
    G_sel2 = G_sel[:,G_sel['snp'].isin(snps)]
    GG = pd.DataFrame(G_sel2.values, columns = G_sel2.snp.values, index = G_sel2.sample.values)
    R = GG.corr()
    R.to_csv(revision_folder+"LD_matrices_for_susie/"+str(gene)+".csv")

ENSG00000164587
ENSG00000134809
ENSG00000197728
ENSG00000214087
ENSG00000198563
ENSG00000163682
ENSG00000130305
ENSG00000170291
ENSG00000146707
ENSG00000254870
ENSG00000243317
ENSG00000226278
ENSG00000165171
ENSG00000189266
ENSG00000140319
ENSG00000132507
ENSG00000147813
ENSG00000186230
ENSG00000204536
ENSG00000148291
ENSG00000233927
ENSG00000204525
ENSG00000165502
ENSG00000204531
ENSG00000204592
ENSG00000133636
ENSG00000172785
ENSG00000228696
ENSG00000076924
ENSG00000169877
ENSG00000229150
ENSG00000141994
ENSG00000204310
ENSG00000100058
ENSG00000213977
ENSG00000172687
ENSG00000152208
ENSG00000204209
ENSG00000108278
ENSG00000145725
ENSG00000170892
ENSG00000243566
ENSG00000116791
ENSG00000100439
ENSG00000172062
ENSG00000196655
ENSG00000183828
ENSG00000164414
ENSG00000112304
ENSG00000196126
ENSG00000172007
ENSG00000248124
ENSG00000142089
ENSG00000125166
ENSG00000175265
ENSG00000137944
ENSG00000111671
ENSG00000168038
ENSG00000117533
ENSG00000206503
ENSG00000180104
ENSG00000178381
ENSG0000

ENSG00000133065
ENSG00000146856
ENSG00000145723
ENSG00000065268
ENSG00000138111
ENSG00000077585
ENSG00000172270
ENSG00000167695
ENSG00000105146
ENSG00000137103
ENSG00000168778
ENSG00000129347
ENSG00000067601
ENSG00000203485
ENSG00000269343
ENSG00000197183
ENSG00000016864
ENSG00000092758
ENSG00000115468
ENSG00000046889
ENSG00000248333
ENSG00000018699
ENSG00000131495
ENSG00000141295
ENSG00000123179
ENSG00000105048
ENSG00000064666
ENSG00000086062
ENSG00000164008
ENSG00000196458
ENSG00000147471
ENSG00000197345
ENSG00000165449
ENSG00000142494
ENSG00000154803
ENSG00000116001
ENSG00000214176
ENSG00000184110
ENSG00000128191
ENSG00000085871
ENSG00000183386
ENSG00000110148
ENSG00000136270
ENSG00000138071
ENSG00000156966
ENSG00000157379
ENSG00000023572
ENSG00000125871
ENSG00000175220
ENSG00000105373
ENSG00000138175
ENSG00000197563
ENSG00000100413
ENSG00000173156
ENSG00000169599
ENSG00000106049
ENSG00000137101
ENSG00000120725
ENSG00000011478
ENSG00000114857
ENSG00000103723
ENSG00000106244
ENSG0000

ENSG00000117519
ENSG00000176125
ENSG00000184209
ENSG00000109686
ENSG00000120306
ENSG00000168813
ENSG00000117305
ENSG00000172663
ENSG00000198498
ENSG00000142168
ENSG00000160949
ENSG00000173166
ENSG00000099821
ENSG00000105696
ENSG00000176261
ENSG00000167005
ENSG00000139746
ENSG00000196365
ENSG00000156261
ENSG00000171735
ENSG00000184979
ENSG00000080815
ENSG00000184752
ENSG00000225697
ENSG00000169174
ENSG00000162669
ENSG00000163958
ENSG00000130713
ENSG00000164096
ENSG00000124782
ENSG00000173406
ENSG00000238083
ENSG00000103351
ENSG00000188976
ENSG00000012963
ENSG00000204922
ENSG00000100764
ENSG00000109133
ENSG00000132541
ENSG00000133028
ENSG00000105223
ENSG00000158941
ENSG00000187097
ENSG00000100263
ENSG00000115216
ENSG00000228409
ENSG00000160345
ENSG00000109084
ENSG00000163879
ENSG00000197147
ENSG00000013810
ENSG00000228782
ENSG00000105866
ENSG00000001617
ENSG00000124374
ENSG00000173638
ENSG00000122547
ENSG00000083844
ENSG00000145088
ENSG00000134108
ENSG00000042317
ENSG00000100304
ENSG0000

ENSG00000166839
ENSG00000166165
ENSG00000196233
ENSG00000141349
ENSG00000110660
ENSG00000165434
ENSG00000188321
ENSG00000154639
ENSG00000116497
ENSG00000119403
ENSG00000235169
ENSG00000170954
ENSG00000105221
ENSG00000165644
ENSG00000196419
ENSG00000106723
ENSG00000142556
ENSG00000182446
ENSG00000196376
ENSG00000125730
ENSG00000087269
ENSG00000182325
ENSG00000131748
ENSG00000188234
ENSG00000083845
ENSG00000086232
ENSG00000168872
ENSG00000005812
ENSG00000105825
ENSG00000164692
ENSG00000124789
ENSG00000114988
ENSG00000212734
ENSG00000164305
ENSG00000163251
ENSG00000163155
ENSG00000137642
ENSG00000147439
ENSG00000119397
ENSG00000197429
ENSG00000171517
ENSG00000111696


In [275]:
i = 0

gene = re.sub("_.*","",genes[i])
gene

'ENSG00000164587'

In [276]:
import os
os.path.exists(revision_folder+"LD_matrices_for_susie/"+str(gene)+".csv")

False

In [277]:
df1 = df0[df0['feature_id']==gene]

In [278]:
df1.shape

(694, 23)

In [279]:
df1.head()

,Unnamed: 0,feature_id,snp_id,p_value,beta,beta_se,empirical_feature_p_value,feature_chromosome,feature_start,feature_end,...,beta_param,rho,snp_chromosome,snp_position,assessed_allele,call_rate,maf,hwe_p,eqtl,z
8384606,15739316,ENSG00000164587,5_149826526_C_T,1.526172e-18,-0.82569,0.093894,0.024524,5,149822753,149829319,...,7.016821e+06,0.9,5,149826526,T,1.0,0.237374,0.411793,ENSG00000164587-5_149826526_C_T,-8.793877
8384608,15739318,ENSG00000164587,5_149821472_T_C,1.062315e-17,-0.82687,0.096454,0.032779,5,149822753,149829319,...,7.016821e+06,0.9,5,149821472,C,1.0,0.232323,0.777426,ENSG00000164587-5_149821472_T_C,-8.572682
8384609,15739319,ENSG00000164587,5_149822090_T_C,1.062315e-17,-0.82687,0.096454,0.032779,5,149822753,149829319,...,7.016821e+06,0.9,5,149822090,C,1.0,0.232323,0.777426,ENSG00000164587-5_149822090_T_C,-8.572682
8384610,15739320,ENSG00000164587,5_149822331_A_G,1.062315e-17,-0.82687,0.096454,0.032779,5,149822753,149829319,...,7.016821e+06,0.9,5,149822331,G,1.0,0.232323,0.777426,ENSG00000164587-5_149822331_A_G,-8.572682
8384611,15739321,ENSG00000164587,5_149822362_G_A,1.062315e-17,-0.82687,0.096454,0.032779,5,149822753,149829319,...,7.016821e+06,0.9,5,149822362,A,1.0,0.232323,0.777426,ENSG00000164587-5_149822362_G_A,-8.572682


In [280]:
df1['feature_id'].unique()

array(['ENSG00000164587'], dtype=object)

In [281]:
snps = df1['snp_id'].unique()
len(snps)

694

In [282]:
G.shape

(1610, 10464962)

In [283]:
G

<xarray.DataArray 'genotype' (sample: 1610, variant: 10464962)>
dask.array<transpose, shape=(1610, 10464962), dtype=float32, chunksize=(1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * sample   (sample) object 'HPSI0216i-tony_5' ... 'HPSI0416pf-cihd'
  * variant  (variant) object '13_13_19020917_G_A' ... '16_16_90274351_C_T'
    fid      (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    iid      (sample) object 'HPSI0216i-tony_5' ... 'HPSI0416pf-cihd'
    father   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    mother   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    gender   (sample) object '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    trait    (sample) object '-9' '-9' '-9' '-9' '-9' ... '-9' '-9' '-9' '-9'
    chrom    (variant) object '13' '13' '13' '13' '13' ... '16' '16' '16' '16'
    snp      (variant) object '13_19020917_G_A' ... '16_90274351_C_T'
    cm       (variant) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    pos      (variant) int32 19020917 19024664 19024785 ... 90193902 90274351
    a0       (variant) object 'G' 'T' 'A' 'G' 'T' 'A' ... 'C' 'T' 'G' 'C' 'C'
    a1       (variant) object 'A' 'C' 'G' 'A' 'C' 'G' ... 'T' 'G' 'T' 'T' 'T'

In [284]:
G_sel = G[:,G['snp'].isin(snps)]
G_sel.shape

(1610, 690)

In [285]:
input_files_dir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/new/input_files/"
sample_mapping_file = input_files_dir+"sample_mapping_file.csv"
sample_mapping = pd.read_csv(sample_mapping_file, dtype={"genotype_individual_id": str, "phenotype_sample_id": str})

In [286]:
samples = sample_mapping['genotype_individual_id'].unique()
len(samples)

126

In [287]:
G_sel2 = G_sel[G_sel['sample'].isin(samples),:]
G_sel2.shape

(125, 690)

In [288]:
## compute correlations (LD matrix)

In [289]:
GG = pd.DataFrame(G_sel2.values, columns = G_sel2.snp.values, index = G_sel2.sample.values)
GG.head()

,5_149572949_C_G,5_149573099_T_C,5_149573365_G_A,5_149573946_T_C,5_149574793_C_A,5_149574837_C_T,5_149574862_G_T,5_149575123_C_T,5_149575702_C_T,5_149575855_G_A,...,5_150035464_C_A,5_150037798_C_T,5_150038065_G_A,5_150038265_G_A,5_150058413_A_G,5_150065019_C_T,5_150065544_T_C,5_150071570_G_C,5_150072693_A_G,5_150078956_A_G
HPSI0115i-zihe_1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
HPSI1014i-eesb_1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
HPSI1014i-quls_2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
HPSI1113i-hajc_1,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0514i-vuna_3,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,2.0,0.0


In [290]:
R = GG.corr()
R.shape

(690, 690)

In [291]:
R.head()

,5_149572949_C_G,5_149573099_T_C,5_149573365_G_A,5_149573946_T_C,5_149574793_C_A,5_149574837_C_T,5_149574862_G_T,5_149575123_C_T,5_149575702_C_T,5_149575855_G_A,...,5_150035464_C_A,5_150037798_C_T,5_150038065_G_A,5_150038265_G_A,5_150058413_A_G,5_150065019_C_T,5_150065544_T_C,5_150071570_G_C,5_150072693_A_G,5_150078956_A_G
5_149572949_C_G,1.000000,0.435272,-0.637428,0.984856,0.330939,0.330939,0.330939,0.852606,0.579415,0.330939,...,-0.062506,0.088102,0.088102,0.088102,-0.111192,0.013325,0.044972,-0.026235,0.009974,0.031452
5_149573099_T_C,0.435272,1.000000,-0.382714,0.422349,-0.058450,-0.058450,-0.058450,0.492954,0.330189,-0.058450,...,-0.074062,-0.134646,-0.134646,-0.134646,-0.120921,-0.066987,-0.038822,-0.084905,-0.122603,0.021983
5_149573365_G_A,-0.637428,-0.382714,1.000000,-0.641503,-0.153807,-0.153807,-0.153807,-0.585872,-0.194734,-0.153807,...,0.074598,-0.108962,-0.108962,-0.108962,-0.025755,-0.020028,-0.030475,0.119031,-0.055732,-0.050961
5_149573946_T_C,0.984856,0.422349,-0.641503,1.000000,0.320551,0.320551,0.320551,0.843075,0.561155,0.320551,...,-0.069884,0.068538,0.068538,0.068538,-0.066281,-0.001843,0.027921,-0.032664,0.015869,0.024625
5_149574793_C_A,0.330939,-0.058450,-0.153807,0.320551,1.000000,1.000000,1.000000,-0.148150,-0.232142,0.935099,...,0.120233,0.162550,0.162550,0.162550,-0.056298,0.130953,0.124610,0.131026,0.074370,0.016604


In [292]:
R.to_csv(revision_folder+"LD_matrices_for_susie/"+str(gene)+".csv")